In [1]:
from dataclasses import dataclass, field
import sys, os
sys.path.insert(1,os.path.abspath('../src'))

Import our code under test.

In [2]:
from fpipeline import pipeline, stepfn, conditionfn, variables

Load our Test framework
    

In [3]:
from tester import Test, set_test_reporter
from notebook_reporter import NotebookTestReporter

set_test_reporter(NotebookTestReporter())

Test our Test framework

In [4]:
Test("Test Test", None).equals(None)

✅: Test Test

In [5]:
Test("Test Negate Test", None).negate.equals(False)

✅: Test Negate Test

#### Test Data Setup

In [6]:
@dataclass
class ctx:
    value: any = None
    trace: list[tuple] = field(default_factory=list)
    def __post_init__(self):
        trace = list()

@stepfn
def tstep(data: ctx, *args, **kwargs):
    d = (*args, {**kwargs})
    data.trace.append(d)
    return data

@conditionfn
def has_value(data: ctx, value):
    return data.value == value

#### Test Cases

In [14]:
Test("@stepfn step takes a single argument",
    tstep(7, 8).__code__.co_argcount
).equals(1)

✅: @stepfn step takes a single argument

In [15]:
Test("@conditionfn condition takes a single argument",
    has_value(7, 8).__code__.co_argcount
).equals(1)

✅: @conditionfn condition takes a single argument

In [9]:
Test("@stepfn capture args",
    tstep(7, 5)(ctx()).trace
).equals([(7, 5, {})])

✅: @stepfn capture args

In [10]:
Test("@conditionfn capture args (true)",
    has_value(7)(ctx(7))
).is_true()

✅: @conditionfn capture args (true)

In [11]:
Test("@conditionfn capture args (false)",
    has_value(7)(ctx())
).is_false()

✅: @conditionfn capture args (false)

In [12]:
Test("Simple pipeline",
    pipeline(tstep(7), tstep(78))(ctx()).trace
 ).equals([(7, {}), (78, {})])

✅: Simple pipeline

In [13]:
def f1():
    with variables(ctx()) as vars:
        return vars.variables
Test("With no variables",
    f1()
).equals({})

✅: With no variables